In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#print(os.listdir("../input"))
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames[:15]: 
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
import cv2
import glob
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

sample = pd.read_csv("/kaggle/input/open-images-object-detection-rvc-2020/sample_submission.csv")
print(sample)
#print(sample.head())
#print(sample.shape)
#print(sample.dtypes)
#print(sample.columns)

#sample.head()

sample = pd.read_csv("/kaggle/input/open-images-object-detection-rvc-2020/sample_submission.csv")
sample.head()

ids = []
for i in range(len(sample)):
    ids.append(sample['ImageId'][i])
print(f"length of ids: {len(ids)}")

#img_data=[]
#for i in range(len(sample)):
#    img_data.append(glob.glob('/kaggle/input/open-images-object-detection-rvc-2020/test/{0}.jpg'.format(ids[i])))


**Create a model**

* Load an SSD model pretrained on COCO dataset.
* Load a a set of utility methods for convenient and comprehensive formatting of input and output of the model.
* Pytorch Hub(torch.hub) is a pre-trained model repository designed to facilitate research reproducibility.
    - torch.hub.load(repo_or_dir, model, *args, source='github', force_reload=False, verbose=True, skip_validation=False, **kwargs)
    - Load a model from a github repo or a local directory.
    - returns the output of the model

In [ ]:
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd')
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

ssd_model.to('cuda')
ssd_model.eval()

New input images for inference

In [ ]:
uris =[
    '/kaggle/input/open-images-object-detection-rvc-2020/test/15fe48f0b21740d2.jpg',
    '/kaggle/input/open-images-object-detection-rvc-2020/test/1e0efd102f2c709c.jpg',
    '/kaggle/input/open-images-object-detection-rvc-2020/test/c67ad5d1d3b72cd5.jpg',
    '/kaggle/input/open-images-object-detection-rvc-2020/test/e57d7ee692d8cf47.jpg',
    '/kaggle/input/open-images-object-detection-rvc-2020/test/1d8aa515a8501a15.jpg'   
]

Format the images to comply with the network input and convert them to tensor

utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')
* Pytorch Hub is a pre-trained model repository designed to facilitate research reproducibility.

In [ ]:
inputs = [utils.prepare_input(uri) for uri in uris]
tensor = utils.prepare_tensor(inputs)
print(inputs)
print(tensor.shape)

Run the SSD network to perform object detection.

**Run the SSD network to perform object detection.**

what is torch.no_grad()?

: Context-manager that disabled gradient calculation.

: Disabling gradient calculation is useful for inference, when you are sure that you will not call Tensor.backward(). It will reduce memory consumption for computations that would otherwise have requires_grad=True.

In [ ]:
with torch.no_grad():
    detections_batch = ssd_model(tensor)
    #print(type(detections_batch))
#print(ssd_model(tensor))

In [ ]:
results_per_input = utils.decode_results(detections_batch)
#print(results_per_input)
best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]

classes_to_labels = utils.get_coco_object_dictionary()
#print(len(classes_to_labels))
#print(classes_to_labels)


In [ ]:
from matplotlib import pyplot as plt
import matplotlib.patches as patches

for image_idx in range(len(best_results_per_input)):
    fig, ax = plt.subplots(1)
    # Show original, denormalized image...
    image = inputs[image_idx] / 2 + 0.5
    ax.imshow(image)
    # ...with detections
    bboxes, classes, confidences = best_results_per_input[image_idx]
    for idx in range(len(bboxes)):
        left, bot, right, top = bboxes[idx]
        x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
plt.show()